In [1]:
# Images
from PIL import Image
from skimage.io import imread
from skimage.transform import resize
import imgaug as aug
import imgaug.augmenters as imaug

# Toolbox
from pathlib import Path
from imutils import paths
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

# Misc
import progressbar
import random
import glob
import sys
import os
import warnings
warnings.filterwarnings(action='ignore')

# Deep learning
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras import applications

# Evaluation
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

Using TensorFlow backend.


In [3]:
# Set directory
IMAGE_FOLDER_PATH = 'C:\\Users\\' + os.getlogin() + '\\Documents\\Programming\\DeepLearning\\09-dog-cat-classifier\\images\\original_train'

FILE_NAMES=os.listdir(IMAGE_FOLDER_PATH)
WIDTH=150
HEIGHT=150

In [4]:
targets=list()
full_paths=list()
for file_name in FILE_NAMES:
    target=file_name.split(".")[0]
    full_path=os.path.join(IMAGE_FOLDER_PATH, file_name)
    full_paths.append(full_path)
    targets.append(target)

dataset=pd.DataFrame()
dataset['image_path']=full_paths
dataset['target']=targets

In [5]:
dataset.head(10)

,image_path,target
0,C:\Users\David Ly\Documents\Programming\DeepLe...,cat
1,C:\Users\David Ly\Documents\Programming\DeepLe...,cat
2,C:\Users\David Ly\Documents\Programming\DeepLe...,cat
3,C:\Users\David Ly\Documents\Programming\DeepLe...,cat
4,C:\Users\David Ly\Documents\Programming\DeepLe...,cat
5,C:\Users\David Ly\Documents\Programming\DeepLe...,cat
6,C:\Users\David Ly\Documents\Programming\DeepLe...,cat
7,C:\Users\David Ly\Documents\Programming\DeepLe...,cat
8,C:\Users\David Ly\Documents\Programming\DeepLe...,cat
9,C:\Users\David Ly\Documents\Programming\DeepLe...,cat


In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense


# path to the model weights files.
top_model_weights_path = 'bottleneck_fc_model_inceptionv3.h5'
# dimensions of our images.
img_width, img_height = 139, 139


train_data_dir = './keras/data_test/train'
validation_data_dir = './keras/data_test/validation'
nb_train_samples = 800
nb_validation_samples = 200
epochs = 10
batch_size = 50

# build the network
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))


# set all layers (312) to non-trainable (weights will not be updated)
for layer in model.layers[:312]:
    layer.trainable = False


# incptv3: 312 -> 0 params
#          311 -> 4,719,105 params



ii=0
for layer in model.layers:
    ii+=1
    print('ii: ',ii,', layer.trainable',layer.trainable)

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

#info a about the layers
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

#more info about the layers
model.summary()

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)



test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
history=model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

print('acc : ', history.history['acc'] )
print('loss: ', history.history['loss'] )